### 3. Duality

#### 3.1 Numerical perturbation analysis example

Consider the quadratic program

\begin{array}{ll}\mbox{minimize} & x_1^2 + 2x_2^2 -x_1 x_2 - x_1\\ \mbox{subject to} & x_1 + 2 x_2 \leq u_1 \\ & x_1 - 4x_2 \leq u_2, \\& x_1+ x_2 \geq -5, \end{array}

with variables $x_1$, $x_2$, and parameters $u_1$, $u_2$.

(a) Solve this QP, for parameter values $u_1=-2$, $u_2=-3$, to find optimal primal variable values $x^\star_1$ and $x_2^\star$, and optimal dual variable values $\lambda_1^\star$, $\lambda_2^\star$ and $\lambda_3^\star$. Let $p^\star$ denote the optimal objective value. <u>Verify that the KKT conditions hold for the optimal primal and dual variables you found</u> (within reasonable numerical accuracy).

We recall that the quadratic form of a product of matrices is:

$$
\begin{bmatrix} 
x_1 & x_2
\end{bmatrix}
\begin{bmatrix} 
a & b \\
c & d 
\end{bmatrix}
\begin{bmatrix} 
x_1  \\
x_2 
\end{bmatrix}
= ax_1^2 + (c+b) \times x_1x_2+dx_2^2
$$

Hint: See $\S$4.7 of the CVX users' guide to find out how to retrieve optimal dual variables. To specify the quadratic objective, use quad_form().

> **What is $x_2^\star$? Enter your result rounded to two decimal places.**

In [ ]:
0.16666667;


> **What is $\lambda_3^\star$? Enter your result rounded to two decimal places.**

In [ ]:
6.60686772272851e-09;

In [ ]:
import cvxpy as cp
import numpy as np
import pandas as pd

# By choosing u_i negative we tighten the inequalities
# Taking the risk that the optimal value of the perturbed problem
# p^* to blow up
x = cp.Variable((2,1))
u1 = cp.Constant(-2)
u2 = cp.Constant(-3)
u3 = cp.Constant(-5)

Q = np.array([[1, -1/2], [-1/2, 2]])
f = np.array([-1, 0])

# Create three constraints
constraints = [x[0] + 2*x[1] <= u1,
               x[0] -4 *x[1] <= u2,
               x[0] + x[1] >= u3]

obj = cp.Minimize(cp.quad_form(x, Q) + np.transpose(f)*x)

# Form and solve problem.
prob = cp.Problem(obj, constraints)
prob.solve()

# The optimal dual variable (Lagrange multiplier) for
# a constraint is stored in constraint.dual_value.
print("optimal x1 + 2*x2 <= u1 \tdual variable", constraints[0].dual_value)
print("optimal x1 -4 *x2 <= u2 \tdual variable", constraints[1].dual_value)
print("optimal x1+x2 >= -5 \tdual variable", constraints[2].dual_value)
print("x value:\t\t", x.value.T)
# Optimal values
print("status:", prob.status)
print("optimal value", prob.value)
# Vector of values generated
print("optimal var", x.value[0:5])

---------

(b) We will now solve some perturbed versions of the QP, with

$$
u_1 = -2 + \delta_1, \qquad u_2 = -3 + \delta_2
$$

where $\delta_1$ and $\delta_2$ each take values from $\{-0.1, 0, 0.1\}$. (There are a total of nine such combinations, including the original problem with $\delta_1=\delta_2=0$.) For each combination of $\delta_1$ and $\delta_2$, make a prediction $p^\star_\mathrm{pred}$ of the optimal value of the perturbed QP, and compare it to $p^\star_\mathrm{exact}$, the exact optimal value of the perturbed QP (obtained by solving the perturbed QP). Find the values that belong in the two righthand columns in a table with the form shown below. Check that the inequality $p^\star_\mathrm{pred} \leq p^\star_\mathrm{exact}$ holds.

\begin{array}{|r||r|r|r|r|}\hline\mathrm{\#}&\delta_1 & \delta_2 & p^\star_\mathrm{pred} & p^\star_\mathrm{exact}\\\hline1&0 & 0 & & \\2&0 & -0.1 & & \\3&0 & 0.1 & & \\\hline4&-0.1 & 0 & & \\5&-0.1 & -0.1 & & \\6&-0.1 & 0.1 & & \\\hline7&0.1 & 0 & & \\8&0.1 & -0.1 & & \\9&0.1 & 0.1 & & \\\hline\end{array}

> **For which perturbations (other than number 1) is $p^\star_\mathrm{exact}-p^\star_\mathrm{pred}$ the smallest?**

Note that the predicted optimal value is given by:
$$
p_{pred}^* = p^* - \lambda_1^* \delta_1 - \lambda_2^* \delta_2
$$

* ```prob.value```is $p^*$
* ```constraints[0].dual_value``` is $\lambda_1$
* ```constraints[1].dual_value``` is $\lambda_2$

In [ ]:
delta1 = [0, 0, 0, -0.1, -0.1, -0.1, 0.1, 0.1, 0.1]
delta2 = [0, -0.1, 0.1, 0, -0.1, 0.1, 0, -0.1, 0.1] 

df = pd.DataFrame({"delta1": delta1, "delta2": delta2,
                   "p*pred": [0]*len(delta1), "p*exact": [prob.value]*len(delta1),
                   "distance": [0]*len(delta1)})
df_temp = df.copy()

for idx, row in df.iterrows():
    # Predicted value
    df_temp.loc[idx, "p*pred"] = prob.value - constraints[0].dual_value * row['delta1']\
                                            - constraints[1].dual_value * row['delta2']
    
    # Exact value
    constraints = [x[0] + 2*x[1] <= u1 + row['delta1'],
                   x[0] -4 *x[1] <= u2 + row['delta2'],
                   -x[0] - x[1] <= -u3]

    obj = cp.Minimize(cp.quad_form(x, Q) + np.transpose(f)*x)

    # Form and solve problem.
    prob = cp.Problem(obj, constraints)
    prob.solve()
    
    df_temp.loc[idx, "p*exact"] = prob.value

In [ ]:
df_temp["distance"] = df_temp["p*exact"] - df_temp["p*pred"]

In [ ]:
df_temp

## A simple example

Consider the optimization problem; note that the constraint is not convex, we cannot solve it directly. We will be using the dual instead

$$
\begin{array}{ll} \mbox{minimize}   & x^2 + 1 \\\mbox{subject to} & (x-2)(x-4) \leq 0,\end{array}
$$

with variable $x \in R$

> **Analysis of primal problem. What is the optimal value ? **

In [ ]:
import holoviews as hv
hv.extension('bokeh')

In [ ]:
x = np.linspace(-5,5,100)
y = x**2 + 1
y_fun = lambda x: x**2+1
lagrangian_fun = lambda lambd: (1+lambd)*(x**2) -6*x*lambd + 1+8*lambd
x_feas_set = np.linspace(2,4,1000)
y_feas_set = 0

In [ ]:
fig_list = dict()
fig_prim = hv.Curve((x, x**2 + 1))
fig_feas = hv.Spread((x_feas_set, y_feas_set, 1000))\
            .options(fill_color='indianred', fill_alpha=.3,line_color='indianred')
fig_sol = hv.Arrow(2,y_fun(2), 'primal_sol', 'v')
fig = fig_feas * fig_sol * fig_prim
for l in np.arange(0,8,0.1):
     fig *= hv.Scatter((x, lagrangian_fun(l)))\
        .options(color='mediumslateblue',size=0.5)
fig.options(width=600, height=600)
# 

## Lagrangian relaxation of Boolean LP

## Option Price Bounds